In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
print(sys.path)

import Models.autoencoders as autoencoders
import Double_Pendulum.Lumped_Mass.robot_parameters as robot_parameters
import Double_Pendulum.Lumped_Mass.transforms as transforms
import Double_Pendulum.Lumped_Mass.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot

from functools import partial



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS
plotter = pendulum_plot.Anim_plotter(rp)
neural_net = True

if neural_net:
    model = autoencoders.Autoencoder_double(rp).to(device)
    model_location = '../Models/Split_AEs/Lumped_Mass_ccw_(2,5).pth'
    model.load_state_dict(torch.load(model_location, weights_only=True))
else:
    model = autoencoders.Analytic_transformer(rp)

In [ ]:
Kp = torch.tensor([[100., 0.], [0., 0.]]).to(device)
Kd = torch.tensor([[40., 0.], [0., 0.]]).to(device)
Kp_old = 50.
Kd_old = 30.

In [ ]:
q_start = torch.tensor([[-torch.pi/2, -torch.pi/2+0.1]]).requires_grad_().to(device)
#q_start = torch.tensor([[-1.5244, -3.0952]]).requires_grad_().to(device)
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
#q_d_start = torch.tensor([[0.5, 0.5]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)

is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))

xy_des_real = torch.tensor([2., -2.]).requires_grad_().to(device)
q_des = transforms.inverse_kinematics(rp, xy_des_real, is_clockwise=False).unsqueeze(0)
#q_des = torch.tensor([[-torch.pi/2, -torch.pi/4]]).requires_grad_().to(device)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

th_start = model.encoder(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T
th_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Obtain accelerations in thetas

is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

th_des = model.encoder(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T

q_des_est = model.decoder(th_des, is_clockwise_des)
q_d_des_est = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
xy_des_est, _ = transforms.forward_kinematics(rp, q_des_est[0])

print(xy_des_est)

In [ ]:
#th_series = th_start.clone().detach()
#th_d_series = th_d_start.clone().detach()
#th_dd_series = th_dd_start.clone().detach()

#q_est_series = model.decoder(th_start, clockwise=is_clockwise_start)
#q_d_est_series = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
#q_dd_est_series = torch.zeros((1,2)).to(device)

#q_real_series = q_start
#q_d_real_series = q_d_start
#q_dd_real_series = q_dd_start

#q_base_series = q_start
#q_d_base_series = q_d_start
#q_dd_base_series = q_dd_start

th_series, th_d_series, th_dd_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_est_series, q_d_est_series, q_dd_est_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_real_series, q_d_real_series, q_dd_real_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_base_series, q_d_base_series, q_dd_base_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)


dt = 0.01
t_end = 15
t_series = torch.arange(0, 15, 0.01)

th = th_start
th_d = th_d_start
th_dd = th_dd_start

q_est = model.decoder(th_start, clockwise=is_clockwise_start)
q_d_est = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
q_dd_est = torch.zeros((1,2))

q_real, q_d_real, q_dd_real = q_start, q_d_start, q_dd_start
q_base, q_d_base, q_dd_base = q_start, q_d_start, q_dd_start

is_clockwise = transforms.check_clockwise(q_start.squeeze(0))

feedforward = False
gravity = True
fix_q = True
model_cw = False
model_shifting = True

if neural_net and fix_q:
    note_string = "NOTE: The controller expects the NN to be trained on " + ("COUNTER" if not model_cw else "") + "CLOCKWISE data."
    print(note_string)
if model_shifting:
    print("NOTE: The controller expects points in q to be shifted (to make q-space unimodal).")



for t in torch.arange(0, t_end, dt):
    print("Time:", t.item().__round__(3))

    if feedforward:
        q = model.decoder(th, is_clockwise)
        q_d = (model.jacobian_dec(th, is_clockwise) @ th_d.T).T
    else:
        is_clockwise = transforms.check_clockwise(q_real.squeeze(0))
        if fix_q and model_cw != is_clockwise:
            print("fixing value to cw")
            q = transforms.flip_q(rp, q_real.squeeze(0), model_cw).unsqueeze(0)
            q_d = transforms.flip_q_d(rp, q_real, q_d_real, model_cw)
        else:
            q = q_real
            q_d = q_d_real

        if model_shifting:
            q = transforms.shift_q(q, clockwise=model_cw)
        th = model.encoder(q)
        th_d = (model.jacobian_enc(q) @ q_d.T).T
    

    J_h = model.jacobian_enc(q)
    J_h_trans = torch.transpose(J_h, 0, 1)

    is_clockwise = transforms.check_clockwise(q.squeeze(0))
    
    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
    A_q = dynamics.input_matrix(rp, q.squeeze(0))


    """ Feed-forward simulation of the system, not on real dynamics """
    A_th = transforms.transform_input_matrix_from_inverse_trans(A_q, J_h_inv_trans, device)
    
    u = torch.pinverse(A_th) @ ((Kp @ (th_des - th).T + Kd @ (th_d_des - th_d).T))
    
    if feedforward:
        M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q, C_q, G_q, th, th_d, J_h_inv, J_h_inv_trans)

        if gravity:
            u = u + torch.pinverse(A_th) @ G_th
        tau_th = A_th * u

        print("u at th:", u)

        if gravity:
            th_dd = (torch.pinverse(M_th) @ (tau_th - C_th @ th_d.T - G_th)).T 
        else:
            th_dd = (torch.pinverse(M_th) @ (tau_th - C_th @ th_d.T)).T

        th = th.detach().requires_grad_()
        th_d = th_d.detach().requires_grad_()
        th_dd = th_dd.detach().requires_grad_()

        th_d = th_d + th_dd * dt
        th = th + th_d * dt

        q = q.detach().requires_grad_()
        q_d = q_d.detach().requires_grad_()
        #q_dd = q_dd.detach().requires_grad_()
        
        J_h_inv_new = model.jacobian_dec(th, is_clockwise).squeeze(0)

        q_d_old = q_d
        q_d = (J_h_inv_new @ th_d.T).T
        q_dd = (q_d - q_d_old)/dt
        q = model.decoder(th, is_clockwise)
        q = wrap_to_pi(q)
    

    """ Update the real system and apply latent control input. """

    M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_real, q_d_real)
    A_q_real = dynamics.input_matrix(rp, q_real)

    if gravity and not feedforward:
        u = u + torch.pinverse(A_q) @ G_q_real

    tau_q_real = A_q_real * u

    if gravity:
        q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T) - G_q_real)).T 
    else:
        q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T))).T
    
    q_d_real = q_d_real + q_dd_real * dt
    q_real = q_real + q_d_real * dt
    q_real = wrap_to_pi(q_real)
    
    if feedforward:
        q_est = q
        q_d_est = q_d
        q_dd_est = q_dd
    else:
        th_est = model.encoder(q_real)
        q_est = model.decoder(th_est, is_clockwise)
        th_d_est = (model.jacobian_enc(q_real) @ q_d_real.T).T
        q_d_est = (model.jacobian_dec(th_est, clockwise=is_clockwise) @ th_d_est.T).T
        q_dd_est = q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Implement with Jacobian derivative


    """ Store data for plotting """

    th_series = torch.cat((th_series, th.detach()), dim=0)
    th_d_series = torch.cat((th_d_series, th_d.detach()), dim=0)
    th_dd_series = torch.cat((th_dd_series, th_dd.detach()), dim=0)
    
    q_real_series = torch.cat((q_real_series, q_real.detach()), dim=0)
    q_d_real_series = torch.cat((q_d_real_series, q_d_real.detach()), dim=0)
    q_dd_real_series = torch.cat((q_dd_real_series, q_dd_real.detach()), dim=0)

    q_est_series = torch.cat((q_est_series, q_est.detach()), dim=0)
    q_d_est_series = torch.cat((q_d_est_series, q_d_est.detach()), dim=0)
    q_dd_est_series = torch.cat((q_dd_est_series, q_dd_est.detach()), dim=0) 

    #print(is_clockwise)
    print("")

    
    
#print(q_real_series)
#print(q_d_real_series)
#print(q_dd_real_series)
#print(q_est_series)
#print(q_d_est_series)
#print(q_dd_est_series)


In [ ]:
stride = 5
pos_end_q_real, pos_elbow_q_real = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_real_series[::stride])
pos_end_q_est, pos_elbow_q_est = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_est_series[::stride])

frames_q_real = plotter.frame_pendulum(pos_end_q_real, pos_elbow_q_real)
frames_q_est = plotter.frame_pendulum(pos_end_q_est, pos_elbow_q_est)

data_real = {
    "frames": frames_q_real,
    "times": dt,
    "name": "q_real", 
    "arm_color": "tab:blue",
    "act_color": "tab:cyan"
}

data_est = {
    "frames": frames_q_est,
    "times": dt,
    "name": "q_est",
    "arm_color": "tab:orange", 
    "act_color": "tab:red"
}

ref_pos_real = {
    "pos": xy_des_real,
    "name": "real",
    "color": "tab:blue"
}

ref_pos_est = {
    "pos": xy_des_est,
    "name": "est",
    "color": "tab:orange"
}

ref_poss = [ref_pos_real, ref_pos_est]

frames_data = [data_real, data_est]
#frames_data = [data_est]
#frames_data = [data_real]

name_base = "(" + str(rp["xa"]) + "," + str(rp["ya"]) + ")_"

if neural_net:
    model_type = "NN"
else:
    model_type = "AL"

if feedforward:
    controller_type = "feedforward" 
else:
    controller_type = "feedback"



file_name = name_base + model_type + "_" + controller_type
if not gravity:
    file_name = file_name + "_no_G"
file_name_gif = file_name + ".gif"
file_name_mp4 = file_name + ".mp4"



plotter.animate_pendulum(frames_data, ref_poss=ref_poss, plot_actuator=True, file_name=file_name_mp4, fps = 1/(dt*stride), dt = dt*stride)
#plotter.animate_pendulum(frames_data, ref_pos=None, plot_actuator=False, file_name=file_name_mp4, fps = 1/(dt*stride), dt = dt*stride)


In [ ]:
datasets_q = [
    {
        "name": "est",
        "values": q_est_series.cpu().detach().numpy(),
        "color": "tab:orange"
    },
    {
        "name": "real",
        "values": q_real_series.cpu().detach().numpy(),
        "color": "tab:blue"
    }#,
    #{
    #    "name": "naive_series",
    #    "values": np.column_stack((np.sin(t_series + 1.0), np.cos(t_series + 1.0))),
    #    "color": "red"
    #}
]
datasets_th = [
    {
        "name": "est",
        "values": th_series.cpu().detach().numpy(),
        "color": "tab:orange"
    }#,56
    #{
    #    "name": "ana",
    #    "values": th_
    #    "color": "tab:blue"
    #}
]

# Common labels for the plots.
name_q = "q trajectory"
name_th = "th trajectory"
t_series = torch.arange(0, 15, 0.01)
# Ensure the results directory exists.
result_path = "./results"
os.makedirs(result_path, exist_ok=True)

# Create an instance of ErrorPlotter.
ep = pendulum_plot.Error_plotter(rp)

# Prepare plot datasets for each column.
# Each call groups a set of datasets to be drawn in one subplot column.
column1 = ep.create_plot_dataset(datasets_q, t_series, name_q)
column2 = ep.create_plot_dataset(datasets_th, t_series, name_th)

# Pass the list of columns (plot_dataset objects) to plot_multi.
ep.plot_multi([column1, column2], file_name="Trajectory_plots", axes_names = ["q", "th"])

In [ ]:
import torch
from torchdiffeq import odeint

class FeedbackODE(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, t, y):
        print("t:", t.item())
        q, q_d = y[0], y[1]

        print("q:", q[0,0].item(), q[0,1].item())

        th = model.encoder(q)
        th_d = (model.jacobian_enc(q) @ q_d.T).T

        J_h = model.jacobian_enc(q)
        J_h_trans = torch.transpose(J_h, 0, 1)

        is_clockwise = transforms.check_clockwise(q.squeeze(0))
        J_h_inv = model.jacobian_dec(th, is_clockwise)
        J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

        M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
        A_q = dynamics.input_matrix(rp, q.squeeze(0))
        #print("M_q:\n", M_q)
        #print("C_q:\n", C_q)
        #print("G_q:\n", G_q)
        A_th = transforms.transform_input_matrix_from_inverse_trans(A_q, J_h_inv_trans, device)
        #print("A_th:\n", A_th)
        u = torch.pinverse(A_th) @ ((Kp @ (th_des - th).T + Kd @ (th_d_des - th_d).T))

        tau_q = A_q @ u
        q_dd = (torch.pinverse(M_q) @ (tau_q - C_q @ q_d.T - G_q)).T
        print("q_dd:", q_dd)
        print("\n")
        return torch.stack([q_d, q_dd])  # Return (dq/dt, dq_d/dt)

# Initial conditions
y0 = torch.stack([q_start, q_d_start])

# Time span for integration
t_span = torch.linspace(0., 1, 10)  # Adjust as needed

# Solve the system
sol = odeint(FeedbackODE(), y0, t_span, method='rk4')  # Use 'bdf' for stiff problems

q_sol, q_d_sol = sol[:, 0, :], sol[:, 1, :]

#print(q_sol)
#print(q_d_sol)


In [ ]:
import warnings
warnings.filterwarnings("ignore")


model_ana = autoencoders.Analytic_transformer(rp)

model_NN = autoencoders.Autoencoder_double(rp).to(device)
model_location = '../Models/Split_AEs/ana_theta_0_loss_quart.pth'
model_NN.load_state_dict(torch.load(model_location, weights_only=True))
models = [model_ana, model_NN]

def check_clockwise_vectorized(q):
    """
    Expects q to be a tensor of shape (N,2) where each row is [q1, q2].
    Returns two boolean masks: (cw_mask, ccw_mask), where:
      - cw_mask[i] is True if the i-th configuration is elbow clockwise.
      - ccw_mask[i] is True if the i-th configuration is elbow counterclockwise.
    
    The logic is as follows (from your original function):
      If q2 lies between q1 and q1+π, or between q1-2π and q1-π, then the configuration
      is considered counterclockwise. Otherwise it is clockwise.
    """
    q1 = q[:, 0]
    q2 = q[:, 1]
    cond_ccw = ((q2 >= q1) & (q2 <= q1 + torch.pi)) | ((q2 >= q1 - 2 * torch.pi) & (q2 <= q1 - torch.pi))
    cw_mask = ~cond_ccw
    ccw_mask = cond_ccw
    return cw_mask, ccw_mask


print("xy_des_real", xy_des_real)
print("xy_des_est", xy_des_est)

# Define the number of grid points along each dimension.
n_points = 200

# Create 1D tensors for q1 and q2 in the range [-pi, 0]
q1_vals = torch.linspace(-np.pi, 0, n_points)
q2_vals = torch.linspace(-np.pi/2, np.pi/2, n_points)

# Create a 2D grid (meshgrid) of q values.
# (Note: using indexing='ij' so that the first axis corresponds to q1 and the second to q2)
q1_grid, q2_grid = torch.meshgrid(q1_vals, q2_vals, indexing='ij')

# Stack the grid to get a tensor of shape (n_points*n_points, 2)
q_grid = torch.stack([q1_grid.flatten(), q2_grid.flatten()], dim=1).to(device)

# === Compute theta1 and theta2 using the analytic encoder functions ===
# We use torch.vmap to evaluate the functions over the batch of q values.
# Note: encoder_theta_1_ana and encoder_theta_2_ana each return a tuple (theta, theta).

for model in models:
  theta_out = model.encoder(q_grid)
  #theta_out = torch.vmap(model.encoder)(q_grid)

  theta1 = theta_out[:, 0]
  theta2 = theta_out[:, 1]

  # Since q1_grid and q2_grid are already on a mesh, we can compute x_end and y_end elementwise.
  x_end = rp["l1"] * torch.cos(q_grid[:, 0]) + rp["l2"] * torch.cos(q_grid[:, 1])
  y_end = rp["l1"] * torch.sin(q_grid[:, 0]) + rp["l2"] * torch.sin(q_grid[:, 1])

  # --- Determine configuration (clockwise vs. counterclockwise) for each q ---
  cw_mask, ccw_mask = check_clockwise_vectorized(q_grid)

  # --- Filter data for each configuration ---
  # Clockwise points
  x_end_cw    = x_end[cw_mask].detach().cpu().numpy()
  y_end_cw    = y_end[cw_mask].detach().cpu().numpy()
  theta1_cw   = theta1[cw_mask].detach().cpu().numpy()
  theta2_cw   = theta2[cw_mask].detach().cpu().numpy()

  # Counterclockwise points
  x_end_ccw   = x_end[ccw_mask].detach().cpu().numpy()
  y_end_ccw   = y_end[ccw_mask].detach().cpu().numpy()
  theta1_ccw  = theta1[ccw_mask].detach().cpu().numpy()
  theta2_ccw  = theta2[ccw_mask].detach().cpu().numpy()

  fig, axes = plt.subplots(2, 2, figsize=(14, 10))

  # --- Top row: Clockwise ---
  sc1 = axes[0, 0].scatter(x_end_cw, y_end_cw, c=theta1_cw, cmap='viridis', s=5)
  axes[0, 0].set_title("Theta1 - Clockwise")
  axes[0, 0].set_xlabel("x")
  axes[0, 0].set_ylabel("y")
  plt.colorbar(sc1, ax=axes[0, 0])

  sc2 = axes[0, 1].scatter(x_end_cw, y_end_cw, c=theta2_cw, cmap='viridis', s=5)
  axes[0, 1].set_title("Theta2 - Clockwise")
  axes[0, 1].set_xlabel("x")
  axes[0, 1].set_ylabel("y")
  plt.colorbar(sc2, ax=axes[0, 1])

  # --- Bottom row: Counterclockwise ---
  sc3 = axes[1, 0].scatter(x_end_ccw, y_end_ccw, c=theta1_ccw, cmap='viridis', s=5)
  axes[1, 0].set_title("Theta1 - Counterclockwise")
  axes[1, 0].set_xlabel("x")
  axes[1, 0].set_ylabel("y")
  plt.colorbar(sc3, ax=axes[1, 0])

  sc4 = axes[1, 1].scatter(x_end_ccw, y_end_ccw, c=theta2_ccw, cmap='viridis', s=5)
  axes[1, 1].set_title("Theta2 - Counterclockwise")
  axes[1, 1].set_xlabel("x")
  axes[1, 1].set_ylabel("y")
  plt.colorbar(sc4, ax=axes[1, 1])

  plt.tight_layout()
  plt.show()


In [ ]:



q = torch.tensor([[torch.pi/4, 0.0]])
q1_in = q[0,0]
q2_in = q[0,1]

q_in_list = []
q_out_cw_list = []
q_out_ccw_list = []

q_in_tensors = torch.empty((0,2))



def theta_from_q(q1_in, q2_in):

    theta1 = torch.sqrt((rp["xa"] - rp["l1"]*torch.cos(q1_in) - rp["l2"]*torch.cos(q2_in))**2 + 
                        (rp["ya"] - rp["l1"]*torch.sin(q1_in) - rp["l2"]*torch.sin(q2_in))**2)

    theta2 = torch.atan2((rp["ya"] - rp["l1"]*torch.sin(q1_in) - rp["l2"]*torch.sin(q2_in)),
                        (rp["xa"] - rp["l1"]*torch.cos(q1_in) - rp["l2"]*torch.cos(q2_in)))
    #print("theta1:", theta1, "   theta2:", theta2)
    return(theta1, theta2)

def q_from_theta(theta1, theta2):
    xend = rp["xa"] - theta1*torch.cos(theta2)
    yend = rp["ya"] - theta1*torch.sin(theta2)


    numerator = (xend**2 + yend**2 - rp["l1"]**2 - rp["l2"]**2)
    denominator = torch.tensor(2*rp["l1"]*rp["l2"])

    beta = torch.arccos(numerator/(denominator+epsilon))
    q1 = torch.atan2(yend, xend + epsilon) - torch.atan2(rp["l2"]*torch.sin(beta), epsilon + rp["l1"] + rp["l2"]*torch.cos(beta))
    q2 = q1 + beta

    q1_alt = torch.atan2(yend, xend) + torch.atan2(rp["l2"]*torch.sin(beta), rp["l1"] + rp["l2"]*torch.cos(beta))
    q2_alt = q1_alt - beta 

    return ((q1, q2), (q1_alt, q2_alt))

def plot_coordinates(q_in_tensors, q_cw, q_ccw):
    """
    Plots three lists of (x, y) coordinates in separate subplots, side by side, with the same limits.

    Parameters:
    - q_in_list: List of (x, y) tuples to be plotted in the first subplot.
    - q_out_list: List of (x, y) tuples to be plotted in the second subplot.
    - q_alt_out_list: List of (x, y) tuples to be plotted in the third subplot.
    """
    # Unpack each list into x and y coordinates
    #x_in, y_in = zip(*q_in_list) if q_in_list else ([], [])
    #x_out, y_out = zip(*q_out_list) if q_out_list else ([], [])
    #x_alt_out, y_alt_out = zip(*q_alt_out_list) if q_alt_out_list else ([], [])

    x_in = q_in_tensors[:, 0].numpy()
    y_in = q_in_tensors[:, 1].numpy()
    x_out = q_cw[:, 0].numpy()
    y_out = q_cw[:, 1].numpy()
    x_alt_out = q_ccw[:, 0].numpy()
    y_alt_out = q_ccw[:, 1].numpy()

    colors = cm.rainbow(np.linspace(0, 1, len(x_in)))

    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

    # Define axis limits
    x_limits = (-1.1*np.pi, 1.1*np.pi)
    y_limits = (-1.1*np.pi, 1.1*np.pi)

    # Plot q_in_list
    axes[0].scatter(x_in, y_in, color=colors, label='q_in_list', alpha=0.7)
    axes[0].set_xlim(x_limits)
    axes[0].set_ylim(y_limits)
    axes[0].set_title('q_in_list')
    axes[0].set_xlabel('q1')
    axes[0].set_ylabel('q2')
    axes[0].grid(True)

    # Plot q_out_list
    axes[1].scatter(x_out, y_out, color=colors, label='q_out_list', alpha=0.7)
    axes[1].set_xlim(x_limits)
    axes[1].set_ylim(y_limits)
    axes[1].set_title('q_alt_out_list')
    axes[1].set_xlabel('X Coordinate')
    axes[1].grid(True)

    # Plot q_alt_out_list
    axes[2].scatter(x_alt_out, y_alt_out, color=colors, label='q_alt_out_list', alpha=0.7)
    axes[2].set_xlim(x_limits)
    axes[2].set_ylim(y_limits)
    axes[2].set_title('q_alt_out_list')
    axes[2].set_xlabel('q1')
    axes[2].grid(True)

    # Adjust layout
    plt.tight_layout()
    plt.show()

epsilon = 0.0001

for j in range(0,1):
    for i in range(0,120):
        q1_in = torch.tensor(i*torch.pi/60 + j*torch.pi/40)
        q2_in = torch.tensor(j*torch.pi/5) - q1_in
        q_in_tensors = torch.cat((q_in_tensors, torch.tensor([[q1_in, q2_in]])), dim=0)

th = torch.vmap(transforms.analytic_theta, in_dims=(None, 0))(rp, q_in_tensors)
q_cw, q_ccw = torch.vmap(transforms.analytic_inverse, in_dims=(None, 0))(rp, th)

plot_coordinates(q_in_tensors, q_cw, q_ccw)




#plot_coordinates(q_in_list, q_out_cw_list, q_out_ccw_list)



for q_in, q_out_cw, q_out_ccw in zip(q_in_tensors, q_cw, q_ccw):

    pos_end, pos_elbow = transforms.forward_kinematics(rp, q_in)
    

    #plotter.plot_double_pendulum([pos_end], [pos_elbow])


pos_end_tensor, pos_elbow_tensor = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_in_tensors)
pos_end_tensor_cw, pos_elbow_tensor_cw = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_cw)
pos_end_tensor_ccw, pos_elbow_tensor_ccw = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_ccw)
colors = cm.rainbow(np.linspace(0, 1, pos_end_tensor.size(0)))
frames_input = plotter.frame_pendulum(pos_end_tensor, pos_elbow_tensor, colors = colors, height=5, width=5)
frames_cw = plotter.frame_pendulum(pos_end_tensor_cw, pos_elbow_tensor_cw, colors = colors, height=5, width=5)
frames_ccw = plotter.frame_pendulum(pos_end_tensor_ccw, pos_elbow_tensor_ccw, colors = colors, height=5, width=5)

print(pos_end_tensor)
print(pos_elbow_tensor)



1. Give setpoint in q-space
2. Give current position in q
3. Map both to theta
4. Calculate distance 